<a href="https://colab.research.google.com/github/Josefh-QM/InteligenceArtificial_NaiveBayes/blob/main/Recrear_un_coche.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reacrear un Coche  
## Plamteamiento del coche
El coche robot es un vehículo autónomo controlado por un microcontrolador Arduino. Está equipado con varios sensores y actuadores para navegar y evitar obstáculos.

## 1. Sensor de Distancia
El sensor de distancia (por ejemplo, un sensor ultrasonido como el HC-SR04) mide la distancia entre el sensor y el objeto más cercano en una dirección específica. Esta información se utiliza para detectar la presencia de obstáculos y determinar su proximidad.

## 2. Posición del Obstáculo
La posición del obstáculo se determina a partir de las lecturas del sensor de distancia. Dependiendo de la configuración del sensor, la posición del obstáculo puede ser identificada en diferentes direcciones:

Delante: Si el sensor está montado en la parte frontal del coche.
Atrás: Si hay un sensor en la parte trasera.
A los lados: Si hay sensores montados a los lados del coche.

## 3. Girar
El coche necesita tomar decisiones basadas en la posición del obstáculo. Las acciones que puede tomar incluyen:

Girar a la izquierda: Si el obstáculo está a la derecha y el coche necesita evitarlo girando hacia la izquierda.
Girar a la derecha: Si el obstáculo está a la izquierda y el coche necesita evitarlo girando hacia la derecha.
Retroceder: Si el coche está demasiado cerca de un obstáculo, puede necesitar retroceder para evitar colisiones.
Avanzar: Si no hay obstáculos en la trayectoria del coche, puede seguir avanzando.

## 4. Dirección
La dirección de movimiento del coche se determina con base en la decisión de giro:

Avanzar: El coche sigue moviéndose hacia adelante.
Girar a la izquierda: El coche gira en la dirección opuesta a la del obstáculo detectado en el lado derecho.
Girar a la derecha: El coche gira en la dirección opuesta a la del obstáculo detectado en el lado izquierdo.
Retroceder: El coche se mueve hacia atrás para evitar el obstáculo.


# SECION METODOLOGICA
## Instancia de la Red Neuronal:

nn = NeuralNetwork([2, 3, 4], activation='tanh'): Crea una red neuronal con 2 neuronas en la capa de entrada, 3 neuronas en la capa oculta y 4 neuronas en la capa de salida. La función de activación utilizada es la tangente hiperbólica (tanh).

## Entrenamiento de la Red Neuronal:

nn.fit(X, y, learning_rate=0.03, epochs=40001): Entrena la red neuronal con los datos X y las etiquetas y. Se usa una tasa de aprendizaje de 0.03 y se realizan 40001 épocas de entrenamiento.

## Función de Redondeo:

def valNN(x): Define una función para redondear los valores predichos y convertirlos a enteros, asegurando que los resultados sean valores discretos.

## Evaluación y Predicción:

for e, target in zip(X, y): Itera sobre cada par de datos de entrada y sus correspondientes etiquetas esperadas.
prediccion = nn.predict(e): Obtiene la predicción de la red neuronal para el dato de entrada e.
print("X:", e, "esperado:", target, "obtenido:", [valNN(p) for p in prediccion]): Imprime el dato de entrada, la etiqueta esperada y la predicción obtenida, redondeando las predicciones a enteros.

In [ ]:
# importacion de librerias necesarias
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np



In [ ]:

# Definición de la clase NeuralNetwork que hereda de torch.nn.Module
class NeuralNetwork(torch.nn.Module):
    # Constructor de la clase
    def __init__(self, layers, activation='tanh'):
        # Llamada al constructor de la clase base
        super(NeuralNetwork, self).__init__()

        # Inicializa una lista para contener las capas de la red
        self.layers = torch.nn.ModuleList()

        # Crea las capas de la red neuronal según la estructura definida en `layers`
        for i in range(1, len(layers)):
            # Añade una capa lineal (totalmente conectada) a la lista de capas
            self.layers.append(torch.nn.Linear(layers[i-1], layers[i]))

        # Selecciona la función de activación en base al parámetro proporcionado
        if activation == 'sigmoid':
            self.activation = torch.nn.Sigmoid()
        elif activation == 'tanh':
            self.activation = torch.nn.Tanh()

    # Método para pasar una entrada a través de la red (propagación hacia adelante)
    def forward(self, x):
        # Aplica cada capa y la función de activación a todas las capas excepto la última
        for layer in self.layers[:-1]:
            x = self.activation(layer(x))
        # Aplica la última capa sin función de activación
        x = self.layers[-1](x)
        return x

    # Método para entrenar la red neuronal
    def fit(self, X, y, learning_rate=0.2, epochs=100000):
        # Define la función de pérdida (error cuadrático medio)
        criterion = torch.nn.MSELoss()
        # Define el optimizador (descenso de gradiente estocástico)
        optimizer = optim.SGD(self.parameters(), lr=learning_rate)

        # Convierte los datos de entrada y salida en tensores de tipo Float
        X = torch.FloatTensor(X)
        y = torch.FloatTensor(y)

        # Bucle de entrenamiento
        for epoch in range(epochs):
            # Pone a cero los gradientes acumulados
            optimizer.zero_grad()
            # Realiza la propagación hacia adelante para obtener las predicciones
            outputs = self(X)
            # Calcula la pérdida comparando las predicciones con los valores reales
            loss = criterion(outputs, y)
            # Calcula los gradientes de la pérdida
            loss.backward()
            # Actualiza los parámetros de la red neuronal
            optimizer.step()

            # Imprime el estado del entrenamiento cada 10000 épocas
            if epoch % 10000 == 0:
                print(f'epoch: {epoch}, loss: {loss.item()}')

    # Método para hacer predicciones con la red neuronal
    def predict(self, x):
        # Desactiva el cálculo de gradientes ya que no es necesario para la predicción
        with torch.no_grad():
            # Convierte la entrada en un tensor de tipo Float
            x = torch.FloatTensor(x)
            # Realiza la propagación hacia adelante y convierte las predicciones a un array de NumPy
            return self(x).numpy()


In [ ]:

# Definición de los datos de entrada (X) para el modelo
X = np.array([
    [-1, 0],   # sin obstáculos
    [-1, 1],   # sin obstáculos
    [-1, -1],  # sin obstáculos
    [0, -1],   # obstáculo detectado a la derecha
    [0, 1],    # obstáculo a la izquierda
    [0, 0],    # obstáculo en el centro
    [1, 1],    # demasiado cerca a la derecha
    [1, -1],   # demasiado cerca a la izquierda
    [1, 0]     # demasiado cerca en el centro
])

# Definición de las etiquetas de salida (y) correspondientes a cada entrada en X
y = np.array([
    [1, 0, 0, 1], # avanzar
    [1, 0, 0, 1], # avanzar
    [1, 0, 0, 1], # avanzar
    [0, 1, 0, 1], # giro a la derecha
    [1, 0, 1, 0], # giro a la izquierda (se cambiaron izquierda y derecha en los comentarios)
    [1, 0, 0, 1], # avanzar
    [0, 1, 1, 0], # retroceder
    [0, 1, 1, 0], # retroceder
    [0, 1, 1, 0]  # retroceder
])


In [ ]:
# Red Coche para Evitar obstáculos
nn = NeuralNetwork([2, 3, 4], activation='tanh')

nn.fit(X, y, learning_rate=0.03, epochs=40001)

def valNN(x):
    return (int)(abs(round(x)))

for e, target in zip(X, y):
    prediccion = nn.predict(e)
    print("X:", e, "esperado:", target, "obtenido:", [valNN(p) for p in prediccion])

epoch: 0, loss: 1.2751439809799194
epoch: 10000, loss: 0.013009816408157349
epoch: 20000, loss: 0.006010393146425486
epoch: 30000, loss: 0.002112950198352337
epoch: 40000, loss: 0.0005535807576961815
X: [-1  0] esperado: [1 0 0 1] obtenido: [1, 0, 0, 1]
X: [-1  1] esperado: [1 0 0 1] obtenido: [1, 0, 0, 1]
X: [-1 -1] esperado: [1 0 0 1] obtenido: [1, 0, 0, 1]
X: [ 0 -1] esperado: [0 1 0 1] obtenido: [0, 1, 0, 1]
X: [0 1] esperado: [1 0 1 0] obtenido: [1, 0, 1, 0]
X: [0 0] esperado: [1 0 0 1] obtenido: [1, 0, 0, 1]
X: [1 1] esperado: [0 1 1 0] obtenido: [0, 1, 1, 0]
X: [ 1 -1] esperado: [0 1 1 0] obtenido: [0, 1, 1, 0]
X: [1 0] esperado: [0 1 1 0] obtenido: [0, 1, 1, 0]


In [ ]:
# Red neuronal para el control de un coche y evitar obstáculos
# Se crea una instancia de la red neuronal con 2 neuronas en la capa de entrada,
# 3 neuronas en la capa oculta y 4 neuronas en la capa de salida. La función de activación usada es 'tanh'.
nn = NeuralNetwork([2, 3, 4], activation='tanh')

# Entrena la red neuronal utilizando los datos de entrada (X) y las etiquetas de salida (y)
# Se utiliza una tasa de aprendizaje de 0.03 y se entrena durante 40001 épocas.
nn.fit(X, y, learning_rate=0.03, epochs=40001)

# Define una función para redondear los valores predichos y convertirlos a enteros
# La función toma un valor, toma el valor absoluto, lo redondea y luego lo convierte a entero.
def valNN(x):
    return (int)(abs(round(x)))

# Itera sobre cada par de datos de entrada (e) y sus correspondientes etiquetas esperadas (target)
for e, target in zip(X, y):
    # Realiza una predicción con la red neuronal para el dato de entrada actual (e)
    prediccion = nn.predict(e)
    # Imprime el dato de entrada, la etiqueta esperada y la predicción obtenida
    # La predicción se redondea y convierte a enteros utilizando la función valNN
    print("X:", e, "esperado:", target, "obtenido:", [valNN(p) for p in prediccion])


epoch: 0, loss: 0.9518530368804932
epoch: 10000, loss: 0.010472545400261879
epoch: 20000, loss: 0.0057090348564088345
epoch: 30000, loss: 0.004176711663603783
epoch: 40000, loss: 0.003119145054370165
X: [-1  0] esperado: [1 0 0 1] obtenido: [1, 0, 0, 1]
X: [-1  1] esperado: [1 0 0 1] obtenido: [1, 0, 0, 1]
X: [-1 -1] esperado: [1 0 0 1] obtenido: [1, 0, 0, 1]
X: [ 0 -1] esperado: [0 1 0 1] obtenido: [0, 1, 0, 1]
X: [0 1] esperado: [1 0 1 0] obtenido: [1, 0, 1, 0]
X: [0 0] esperado: [1 0 0 1] obtenido: [1, 0, 0, 1]
X: [1 1] esperado: [0 1 1 0] obtenido: [0, 1, 1, 0]
X: [ 1 -1] esperado: [0 1 1 0] obtenido: [0, 1, 1, 0]
X: [1 0] esperado: [0 1 1 0] obtenido: [0, 1, 1, 0]
